In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
import time
from datetime import date
from bs4 import BeautifulSoup, Comment
import math

In [8]:
# This will allow us to scrape baseball almanac 
HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

# Set up methods for updating baseball almanac's url for every option
alphabet_list = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
number_counter = 0
alphabet_counter = 0
players_list = []


while alphabet_counter < 26:        
        
    if number_counter == 0:
        sub_counter = ""
    else:
        sub_counter = number_counter
    
    # Checking which URLs are being scraped
    print(f'https://www.baseball-almanac.com/players/player-{alphabet_list[alphabet_counter]}{sub_counter}.shtml')
    
    r = requests.get(f'https://www.baseball-almanac.com/players/player-{alphabet_list[alphabet_counter]}{sub_counter}.shtml', headers=HEADERS)
          
    # Creating a way for the loop to continue once we meet all options for a specific letter
    if str(r) == "<Response [404]>":
        
        try:
            sub_counter = sub_counter + 1
        except TypeError:
            alphabet_counter = alphabet_counter + 1
            number_counter = 0
            continue
        
        r = requests.get(f'https://www.baseball-almanac.com/players/player-{alphabet_list[alphabet_counter]}{sub_counter}.shtml', headers=HEADERS)
        
        if str(r) == "<Response [404]>":
            alphabet_counter = alphabet_counter + 1
            number_counter = 0
            continue
            
    # initializing substrings
    sub2 = "td>"
    sub1 = "<td"

    cleaned_sub1=str(re.escape(sub1))
    cleaned_sub2=str(re.escape(sub2))

    # grabbing names and years played from scraped info
    res=re.findall(cleaned_sub1+"(.*)"+cleaned_sub2,r.text)

    sub3 = "Show stats for "
    sub4 = "</a>"

    for i in range(len(res)):
        try:
            a=str(re.escape(sub3))
            b=str(re.escape(sub4))

            # printing result
            test=re.findall(a+"(.*)"+b,res[i])
            test3 = re.findall("\d{4}",res[i])
            test_list = [test[0].split(">",1)[1],test3[0],test3[1]]
            
            if len(test3) == 3:
                test_list.append(test3[2])
            else:
                test_list.append("NO")
                
            players_list.append(test_list)
        except:
            continue
                
    number_counter = number_counter + 1

# cleaning remaining names that have html code included as text in the name
for i in range(len(players_list)):
    if "</a>" in players_list[i][0]:
        temp_item = players_list[i][0].split("</a>")
        players_list[i][0] = temp_item[0]

        
players_df = pd.DataFrame(players_list, columns = ["Name", "Career Start", "Career End","HOF"])
players_df

https://www.baseball-almanac.com/players/player-a.shtml
https://www.baseball-almanac.com/players/player-a1.shtml
https://www.baseball-almanac.com/players/player-a2.shtml
https://www.baseball-almanac.com/players/player-a3.shtml
https://www.baseball-almanac.com/players/player-b.shtml
https://www.baseball-almanac.com/players/player-b1.shtml
https://www.baseball-almanac.com/players/player-b2.shtml
https://www.baseball-almanac.com/players/player-b3.shtml
https://www.baseball-almanac.com/players/player-b4.shtml
https://www.baseball-almanac.com/players/player-b5.shtml
https://www.baseball-almanac.com/players/player-b6.shtml
https://www.baseball-almanac.com/players/player-b7.shtml
https://www.baseball-almanac.com/players/player-b8.shtml
https://www.baseball-almanac.com/players/player-c.shtml
https://www.baseball-almanac.com/players/player-c1.shtml
https://www.baseball-almanac.com/players/player-c2.shtml
https://www.baseball-almanac.com/players/player-c3.shtml
https://www.baseball-almanac.com/p

,Name,Career Start,Career End,HOF
0,David Aardsma,2004,2015,NO
1,Hank Aaron,1954,1976,1982
2,Tommie Aaron,1962,1971,NO
3,Don Aase,1977,1990,NO
4,Andy Abad,2001,2006,NO
...,...,...,...,...
24518,Frank Zupo,1957,1961,NO
24519,Paul Zuvella,1982,1991,NO
24520,George Zuverink,1951,1959,NO
24521,Dutch Zwilling,1910,1916,NO


In [9]:
players_df.to_csv('all_MLB_players.csv')
